# Hate Speech Detector 2.0
---
**Initial data analysis**
1. Selection of relevant tweet data.
    1. {'new_id', 'date', 'time', 'user_id', 'username', 'name', 'tweet', 'emojis', 'emoticons', 'mentions'}
    2. {'hashtags', 'reply_to', 'replies_count', 'retweets_count', 'likes_count'}
2. Combining selected data with their annotations and saving into separete files.
3. Cardinalities and combination od classes counting + visualization.
4. Hateful phrases analysis:
    1. Set of raw hateful phrases
    2. Set of lemmatized hateful phrases
    3. Set of synonymic hateful phrases
    4. Calculation of phrases appearance is text 
        1. Does appear fully or partially, how?
        2. Get max and mean values.
        3. 1.0 means hate speech --> 0.0 mean no hate speech
5. For each of 7 hate speech classes:
    1. Load appropriate .txt file with hateful lemmatized phrases.
    2. Load appropriate .txt file with synonymic hateful phrases.
    3. For each lemmatized tweet:
        1. Calculate min, mean and max PAC (Phrase Appearance Coefficient) scores.
        2. Get means of mins, means and maxes.
6. Polish polyglot sentiment analysis
7. Characters, syllables, words counting.
8. For each tweet:
    1. Determine how many words have which type of sentiment.
    2. Count characters, syllables, words and unique words.
9. For each of 7 hate speech classes and one vulgar:
    1. Detect 10 hateful topics which include 10 words.
    2. Save LDA model.
    3. For each tweet:
        1. Calculate PAC scores of topics appearance and mean aggregate over topics

In [1]:
import numpy as np
import pandas as pd

import itertools

import morfeusz2
from pyplwnxml import PlwnxmlParser

from polyglot.text import Text
from polyglot.downloader import downloader

import pyphen

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation as LDA

from tqdm.notebook import tqdm

import os
import csv
import pickle

In [2]:
from polyglot.detect.base import logger as polyglot_logger
polyglot_logger.setLevel("ERROR")

In [3]:
pd.set_option('display.max_colwidth', 400)

**Polish stopwords**

In [4]:
with open('data/other/polish_stopwords.txt', 'r') as f:
    polish_stopwords = f.read().split('\n')[:-1]
polish_stopwords[:10]

['a', 'aby', 'ach', 'acz', 'aczkolwiek', 'aj', 'albo', 'ale', 'alez', 'ależ']

## Selection of relevant data

In [5]:
df_infos = pd.read_csv('data/sady_main/sady_infos_sanitized.csv')
df_infos.head(2)

/home/jakub/Dokumenty/venv36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (9) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,id,conversation_id,created_at,date,time,timezone,user_id,username,name,place,...,source,user_rt_id,user_rt,retweet_id,reply_to,retweet_date,translate,trans_src,trans_dest,new_id
0,"4,74999593189773E+017","4,74982910425301E+017",1402083702000,2014-06-06,21:41:42,CEST,2367137142,krzysztofcicho3,Krzysztof Cichosz,NaN,...,NaN,NaN,NaN,NaN,"[{'user_id': '2367137142', 'username': 'KrzysztofCicho3'}, {'user_id': '244246777', 'username': 'TomSokolewicz'}]",NaN,NaN,NaN,NaN,0
1,"4,74958618094105E+017","4,74958618094105E+017",1402073933000,2014-06-06,18:58:53,CEST,780543464,zalewski53,Roland Zalewski,NaN,...,NaN,NaN,NaN,NaN,"[{'user_id': '780543464', 'username': 'Zalewski53'}]",NaN,NaN,NaN,NaN,1


In [6]:
df_infos.columns

Index(['id', 'conversation_id', 'created_at', 'date', 'time', 'timezone',
       'user_id', 'username', 'name', 'place', 'tweet', 'emojis', 'emoticons',
       'mentions', 'urls', 'photos', 'replies_count', 'retweets_count',
       'likes_count', 'hashtags', 'cashtags', 'link', 'retweet', 'quote_url',
       'video', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id',
       'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest',
       'new_id'],
      dtype='object')

In [7]:
df_annotated = pd.read_csv('data/sady_main/sady_date_annotated.csv', sep='\t')
df_annotated = df_annotated.drop(columns=['date', 'time', 'tekst', 'inne', 'inne.1'])
df_annotated = df_annotated.drop(columns=['ksenofobia', 'szowinizm', 'rasizm', 'seksizm',
                                          'antysemityzm', 'homofobia'])
df_annotated.head()

,id,wydźwięk,klucze,wyzywanie,grożenie,wykluczanie,odczłowieczanie,poniżanie,stygmatyzacja,szantaż
0,9,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,8,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,7,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,3,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
relevant_cols = ['new_id', 'date', 'time', 'user_id', 'username', 'name', 'tweet', 'emojis', 'emoticons',
                 'mentions', 'hashtags', 'reply_to', 'replies_count', 'retweets_count', 'likes_count']

df_infos = df_infos[relevant_cols]
df_infos['new_id'] = df_infos['new_id'].astype(int)
df_infos.head(2)

,new_id,date,time,user_id,username,name,tweet,emojis,emoticons,mentions,hashtags,reply_to,replies_count,retweets_count,likes_count
0,0,2014-06-06,21:41:42,2367137142,krzysztofcicho3,Krzysztof Cichosz,Dokładnie! Dlatego trzeba komuchów gonić przed sądy póki żyją. I mięć otwarte oczy na komuchów zakamuflowanych,NaN,NaN,['tomsokolewicz'],[],"[{'user_id': '2367137142', 'username': 'KrzysztofCicho3'}, {'user_id': '244246777', 'username': 'TomSokolewicz'}]",0,0,0
1,1,2014-06-06,18:58:53,780543464,zalewski53,Roland Zalewski,Polska Polityka: Sądy bardziej bezkarne niż w PRL,NaN,NaN,[],[],"[{'user_id': '780543464', 'username': 'Zalewski53'}]",0,0,0


## Combining data

In [9]:
df_combined = df_infos.merge(df_annotated, left_on='new_id', right_on='id')
df_combined = df_combined.drop(columns=['new_id'])
df_combined.head(2)

,date,time,user_id,username,name,tweet,emojis,emoticons,mentions,hashtags,...,id,wydźwięk,klucze,wyzywanie,grożenie,wykluczanie,odczłowieczanie,poniżanie,stygmatyzacja,szantaż
0,2014-06-06,21:41:42,2367137142,krzysztofcicho3,Krzysztof Cichosz,Dokładnie! Dlatego trzeba komuchów gonić przed sądy póki żyją. I mięć otwarte oczy na komuchów zakamuflowanych,NaN,NaN,['tomsokolewicz'],[],...,0,-1,komuchów; gonić komuchów,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,2014-06-06,18:58:53,780543464,zalewski53,Roland Zalewski,Polska Polityka: Sądy bardziej bezkarne niż w PRL,NaN,NaN,[],[],...,1,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
df_combined.columns

Index(['date', 'time', 'user_id', 'username', 'name', 'tweet', 'emojis',
       'emoticons', 'mentions', 'hashtags', 'reply_to', 'replies_count',
       'retweets_count', 'likes_count', 'id', 'wydźwięk', 'klucze',
       'wyzywanie', 'grożenie', 'wykluczanie', 'odczłowieczanie', 'poniżanie',
       'stygmatyzacja', 'szantaż'],
      dtype='object')

In [11]:
df_combined = df_combined[['id', 'date', 'time', 'user_id', 'username', 'name', 'tweet', 'emojis', 'emoticons',
                           'mentions', 'hashtags', 'reply_to', 'replies_count', 'retweets_count', 'likes_count',
                           'wydźwięk', 'klucze', 'wyzywanie', 'grożenie', 'wykluczanie',
                           'odczłowieczanie', 'poniżanie', 'stygmatyzacja', 'szantaż']]
df_combined.head(2)

,id,date,time,user_id,username,name,tweet,emojis,emoticons,mentions,...,likes_count,wydźwięk,klucze,wyzywanie,grożenie,wykluczanie,odczłowieczanie,poniżanie,stygmatyzacja,szantaż
0,0,2014-06-06,21:41:42,2367137142,krzysztofcicho3,Krzysztof Cichosz,Dokładnie! Dlatego trzeba komuchów gonić przed sądy póki żyją. I mięć otwarte oczy na komuchów zakamuflowanych,NaN,NaN,['tomsokolewicz'],...,0,-1,komuchów; gonić komuchów,1.0,NaN,NaN,NaN,NaN,NaN,NaN
1,1,2014-06-06,18:58:53,780543464,zalewski53,Roland Zalewski,Polska Polityka: Sądy bardziej bezkarne niż w PRL,NaN,NaN,[],...,0,-1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
df_combined.to_csv('data/sady_main/sady_combined.csv', index=False)

## Class cardinalities

In [13]:
df_classes = df_combined[['wyzywanie', 'grożenie', 'wykluczanie', 'odczłowieczanie',
                          'poniżanie', 'stygmatyzacja', 'szantaż']]
df_classes = df_classes.notnull().astype('int')
df_classes['number'] = df_classes.index
df_classes.head(2)

,wyzywanie,grożenie,wykluczanie,odczłowieczanie,poniżanie,stygmatyzacja,szantaż,number
0,1,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,1


**GENERALLY** How many examples belong to each class?

In [14]:
srs_classes = df_classes.sum().sort_values(ascending=False)
srs_classes['number'] = df_classes['number'].count()
srs_classes

number             15202
stygmatyzacja        830
poniżanie            700
grożenie             393
wyzywanie            242
odczłowieczanie      174
wykluczanie           94
szantaż                6
dtype: int64

In [15]:
pd.Series(srs_classes/len(df_classes)*100).sort_values(ascending=False)

number             100.000000
stygmatyzacja        5.459808
poniżanie            4.604657
grożenie             2.585186
wyzywanie            1.591896
odczłowieczanie      1.144586
wykluczanie          0.618340
szantaż              0.039468
dtype: float64

The 'stygmatyzacja' (5.45%) and 'poniżanie' (4.60%) labels it's at most.

**SPECIFICALLY** How many examples belong to each combination of classes?

In [16]:
dfagg_classes = df_classes.groupby(['wyzywanie', 'grożenie', 'wykluczanie', 'odczłowieczanie',
                                  'poniżanie', 'stygmatyzacja', 'szantaż'])\
                                  .count().sort_values(by='number', ascending=False)
dfagg_classes['%'] = dfagg_classes['number']/len(df_classes)*100
dfagg_classes

number  \
wyzywanie grożenie wykluczanie odczłowieczanie poniżanie stygmatyzacja szantaż           
0         0        0           0               0         0             0         13654   
                                                         1             0           361   
                                               1         0             0           251   
                                                         1             0           197   
          1        0           0               0         0             0           179   
                                                         1             0           106   
1         0        0           0               1         0             0            57   
                                               0         0             0            43   
                               1               1         1             0            40   
0         0        0           1               0         0             0            36   
          1        1           0               0         0             0            32   
1         0        0           0               1         1             0            32   
0         0        0           1               1         0             0            23   
          1        0           0               1         0             0            20   
1         0        0           0               0         1             0            20   
                               1               1         0             0            15   
0         0        1           0               0         0             0            13   
                   0           1               0         1             0            13   
                                               1         1             0            11   
                   1           0               1         0             0            10   
          1        0           0               1         1             0             8   
1         0        0           1               0         1             0             7   
0         0        1           0               0         1             0             6   
          1        0           1               0         1             0             6   
1         1        0           0               0         0             0             5   
0         1        0           1               1         0             0             5   
1         1        0           0               1         0             0             4   
          0        1           0               1         1             0             4   
0         1        1           0               0         1             0             4   
                   0           1               0         0             0             4   
                   1           0               1         1             0             3   
          0        0           0               0         0             1             3   
          1        0           1               1         1             0             3   
          0        1           0               1         1             0             3   
          1        1           1               0         0             0             2   
1         1        1           0               1         0             0             2   
                                               0         0             0             2   
          0        1           1               1         0             0             2   
0         0        1           1               1         0             0             2   
          1        0           0               0         0             1             2   
1         0        1           0               1         0             0             1   
                                               0         0             0             1   
0         0        1           1               1         1             0             1   
1         0        1          

The combinations of labels single 'stygmatyzacja' (2.37%) and 'stygmatyzacja' with 'poniżanie' (1.65%) it's at most

## Hateful phrases analysis

In [17]:
df_phrases = df_combined[['klucze', 'wyzywanie', 'grożenie', 'wykluczanie',
                          'odczłowieczanie', 'poniżanie', 'stygmatyzacja', 'szantaż']]
df_phrases = df_phrases.notnull().astype('int')
df_phrases['klucze'] = df_combined['klucze']
df_phrases = df_phrases.dropna()
df_phrases['klucze'] = list([phr.replace('[..]', '[...]') for phr in df_phrases['klucze']])

df_phrases.head()

,klucze,wyzywanie,grożenie,wykluczanie,odczłowieczanie,poniżanie,stygmatyzacja,szantaż
0,komuchów; gonić komuchów,1,0,0,0,0,0,0
2,odbierajmy [...] bałwanom,0,0,0,0,1,0,0
19,spocona świnia; świnia,0,0,0,0,1,0,0
41,durne angielskie przepisy,0,0,0,0,1,0,0
48,Kwaśniewski [...] idiota,1,0,0,0,1,0,0


Convert list-like column elements to separate rows: [link](https://cmdlinetips.com/2020/06/pandas-explode-convert-list-like-column-elements-to-separate-rows/)

In [18]:
df_phrases['klucze'] = list([k.split(';') for k in df_phrases['klucze']])
df_phrases = df_phrases.explode('klucze')
df_phrases.head()

,klucze,wyzywanie,grożenie,wykluczanie,odczłowieczanie,poniżanie,stygmatyzacja,szantaż
0,komuchów,1,0,0,0,0,0,0
0,gonić komuchów,1,0,0,0,0,0,0
2,odbierajmy [...] bałwanom,0,0,0,0,1,0,0
19,spocona świnia,0,0,0,0,1,0,0
19,świnia,0,0,0,0,1,0,0


### Raw

In [19]:
phr_wyz = list(df_phrases[df_phrases['wyzywanie'] == 1]['klucze'])
phr_wyz[:5]

['komuchów',
 ' gonić komuchów',
 'Kwaśniewski [...] idiota',
 'pojebało kogoś',
 'politycy [...] kłapania dziobem']

In [20]:
phr_groz = list(df_phrases[df_phrases['grożenie'] == 1]['klucze'])
phr_groz[:5]

['sądy [...] skasują',
 'bomb[...] domy podejrzanych',
 'policzek [...] nie zaszkodzi',
 'że was zajebiemy',
 'sądy gówniane']

In [21]:
phr_wyk = list(df_phrases[df_phrases['wykluczanie'] == 1]['klucze'])
phr_wyk[:5]

['ryj [...] zakazany',
 'prezydent nie wart szacunku',
 ' Premiar morderca, zdrajca',
 'wypierdoliliśmy ją z roboty',
 ' nie ma kasy na sądy [...] nam nie skoczy']

In [22]:
phr_odcz = list(df_phrases[df_phrases['odczłowieczanie'] == 1]['klucze'])
phr_odcz[:5]

['komisja śledcza z pachołkami bez wiedzy',
 'prezydent nie wart szacunku',
 ' Premiar morderca, zdrajca',
 'bomb[...] domy podejrzanych',
 'esbeckie złogi']

In [23]:
phr_pon = list(df_phrases[df_phrases['poniżanie'] == 1]['klucze'])
phr_pon[:5]

['odbierajmy [...] bałwanom',
 'spocona świnia',
 ' świnia',
 'durne angielskie przepisy',
 'Kwaśniewski [...] idiota']

In [24]:
phr_styg = list(df_phrases[df_phrases['stygmatyzacja'] == 1]['klucze'])
phr_styg[:5]

['sądy [...] to zbytek',
 ' #Józefmoneta',
 'PiS gwałci żeby nie robić aborcji',
 'ryj [...] zakazany',
 'politycy [...] kłapania dziobem']

In [25]:
phr_szan = list(df_phrases[df_phrases['szantaż'] == 1]['klucze'])
phr_szan[:5]

['wypierdalaj stąd, bo zaliczysz zgon',
 ' jesteś kurwą, wypierdalaj',
 ' wypierdalaj',
 'jak nie odsuniecie [...] to zamkniemy was w pierdlu',
 'jak dalej będziesz tworzyć trudne sądy [...] przez Ciebie przejdę ząłamanie nerwowe']

### Lemmatized

In [26]:
morf = morfeusz2.Morfeusz()

In [27]:
def lemmatize_text(text):
    analysis = morf.analyse(text.replace('#', ''))
    lemmas = list([])
    
    i, j, interp = analysis[0]
    last_ij, last_lemma = (i, j), interp[1].split(':')[0].lower()
    lemmas.append(last_lemma)
    
    for i, j, interp in analysis[1:]:
        lemma = interp[1].split(':')[0].lower()
        if not (last_ij == (i, j) and last_lemma == lemma):
            lemmas.append(lemma)
        
        last_ij = (i, j)
        last_lemma = lemma
    
    lemm_text = ' '.join(lemmas)
    lemm_text = lemm_text.replace(' [ . . . ]', '')
    
    return lemm_text

def get_lemmatized_phrases(phrases, save_file=None):
    lemm_phrases = list([])
    
    for phrase in phrases:
        lemm_phrase = lemmatize_text(phrase)
        if lemm_phrase not in lemm_phrases:
            lemm_phrases.append(lemm_phrase)
    
    if save_file:
        with open(save_file, 'w') as f:
            for lemm_phrase in lemm_phrases:
                f.writelines(lemm_phrase + '\n')
    
    return lemm_phrases

In [28]:
lemmatize_text('sądy cycki zrobiła ja pierdolę')

'sąd cycek zrobić ja pierdolić'

In [29]:
lemmphr_wyz = get_lemmatized_phrases(phr_wyz, save_file='data/hateful_phrases/lemm_wyz.txt')
lemmphr_wyz[:5]

['komuch',
 'gonić komuch',
 'kwaśniewski idiota',
 'pojebać kto ktoś być',
 'polityk kłapać dziób']

In [30]:
lemmphr_groz = get_lemmatized_phrases(phr_groz, save_file='data/hateful_phrases/lemm_groz.txt')
lemmphr_groz[:5]

['sąd skasować',
 'bomba dom podejrzeć podejrzany podejrzana',
 'policzek nie on nie zaszkodzić',
 'że wy zajebać',
 'sąd gówniany']

In [31]:
lemmphr_wyk = get_lemmatized_phrases(phr_wyk, save_file='data/hateful_phrases/lemm_wyk.txt')
lemmphr_wyk[:5]

['ryj ryć zakazać zakazany',
 'prezydent nie on nie wart warta wart szacunek',
 'premiara morderca , zdrajca',
 'wypierdolić być on z roboty robot robota',
 'nie on nie mój mieć kasa na sąd my nie on nie skoczyć']

In [32]:
lemmphr_odcz = get_lemmatized_phrases(phr_odcz, save_file='data/hateful_phrases/lemm_odcz.txt')
lemmphr_odcz[:5]

['komisja śledczy z pachołek bez beza wiedza',
 'prezydent nie on nie wart warta wart szacunek',
 'premiara morderca , zdrajca',
 'bomba dom podejrzeć podejrzany podejrzana',
 'esbecki złóg']

In [33]:
lemmphr_pon = get_lemmatized_phrases(phr_pon, save_file='data/hateful_phrases/lemm_pon.txt')
lemmphr_pon[:5]

['odbierać bałwan',
 'spocić świnia świni',
 'świnia świni',
 'durny angielski przepis',
 'kwaśniewski idiota']

In [34]:
lemmphr_styg = get_lemmatized_phrases(phr_styg, save_file='data/hateful_phrases/lemm_styg.txt')
lemmphr_styg[:5]

['sąd ten to zbytek',
 'józefmoneta',
 'pis pisa gwałcić żeby nie on nie robić aborcja',
 'ryj ryć zakazać zakazany',
 'polityk kłapać dziób']

In [35]:
lemmphr_szan = get_lemmatized_phrases(phr_szan, save_file='data/hateful_phrases/lemm_szan.txt')
lemmphr_szan[:5]

['wypierdalać stąd , bo zaliczyć zgon',
 'być kurwa , wypierdalać',
 'wypierdalać',
 'jak jaka jak nie on nie odsunąć ten to zamknąć wy w pierdel',
 'jak jaka jak daleko dalej być tworzyć trudny sąd przez ty przejść ząłamanie nerwowy']

### Synonymic
Get negative or neutral sentiment synonyms for each lemmatized word and perform cartesian product over listed lists of word synonyms.

In [36]:
pl_wordnet = PlwnxmlParser('models/plwordnet_3_0/plwordnet-3.0.xml').read_wordnet()

In [37]:
def synonymic_phrases(phrase, lemmatized=True, stopwords=[]):
    synonymic_phrases_options = list([])
    p = phrase if lemmatized else lemmatize_text(phrase)
    
    lemm_words = list([w for w in p.split(' ') if w not in stopwords])
    
    for lemm_word in lemm_words:
        lemm_word_options = list([])
        for lemm in pl_wordnet.lemma(lemm_word):
            for synset in lemm.synsets:
                for lu in synset.lexical_units:
                    if lu.sentiment in ['- m', '- s'] and lu.name not in lemm_word_options:
                        lemm_word_options.append(lu.name)
        
        if len(lemm_word_options) == 0:
            lemm_word_options.append(lemm_word)
        synonymic_phrases_options.append(lemm_word_options)
    
    options = list(itertools.product(*synonymic_phrases_options))
    options = list([' '.join(option) for option in options])
    if p in options:
        options.remove(p)
    
    return options

def get_synonymic_phrases(phrases, lemmatized=True, stopwords=[], save_file=None):
    all_syn_phrases = list([])
    
    for phrase in phrases:
        syn_phrases = synonymic_phrases(phrase, lemmatized=lemmatized, stopwords=stopwords)
        for sp in syn_phrases:
            if sp not in all_syn_phrases:
                all_syn_phrases.append(sp)
    
    if save_file:
        with open(save_file, 'w') as f:
            for syn_phrase in all_syn_phrases:
                f.writelines(syn_phrase + '\n')
    
    return all_syn_phrases

In [38]:
synonymic_phrases('faszyzm sąd faszyzm sąd', stopwords=polish_stopwords)

[]

In [39]:
synphr_wyz = get_synonymic_phrases(lemmphr_wyz, stopwords=polish_stopwords,
                                   save_file='data/hateful_phrases/syn_wyz.txt')
synphr_wyz[:5]

['pojebać',
 'czuj czuja czuć ?',
 'prezydent wart warta wart szacunek',
 'ch . m miasto morze męski metr . wiek wielki wiersz wieś wyspa . d dawny dom dzień .',
 'chuj dupa']

In [40]:
synphr_groz = get_synonymic_phrases(lemmphr_groz, stopwords=polish_stopwords,
                                    save_file='data/hateful_phrases/syn_groz.txt')
synphr_groz[:5]

['policzek zaszkodzić',
 'zajebać',
 'ruski ruskie ruski wypieprzyć',
 'prezydent czarna czarny dupa',
 'wyczyścić sędzia']

In [41]:
synphr_wyk = get_synonymic_phrases(lemmphr_wyk, stopwords=polish_stopwords,
                                   save_file='data/hateful_phrases/syn_wyk.txt')
synphr_wyk[:5]

['prezydent wart warta wart szacunek',
 'wypierdolić roboty robot robota',
 'mieć kasa sąd skoczyć',
 'ruski ruskie ruski wypieprzyć',
 'wyczyścić sędzia']

In [42]:
synphr_odcz = get_synonymic_phrases(lemmphr_odcz, stopwords=polish_stopwords,
                                    save_file='data/hateful_phrases/syn_odcz.txt')
synphr_odcz[:5]

['komisja śledczy pachołek beza wiedza',
 'prezydent wart warta wart szacunek',
 'esbecki złóg / psl',
 'sąd cycek zrobić pierdolić',
 'chcieć menda']

In [43]:
synphr_pon = get_synonymic_phrases(lemmphr_pon, stopwords=polish_stopwords,
                                   save_file='data/hateful_phrases/syn_pon.txt')
synphr_pon[:5]

['sąd zbytek',
 'pis pisa gwałcić robić aborcja',
 'komisja śledczy pachołek beza wiedza',
 'czuj czuja czuć ?',
 'prezydent wart warta wart szacunek']

In [44]:
synphr_styg = get_synonymic_phrases(lemmphr_styg, stopwords=polish_stopwords,
                                    save_file='data/hateful_phrases/syn_styg.txt')
synphr_styg[:5]

['sąd zbytek',
 'pis pisa gwałcić robić aborcja',
 'prezydent wart warta wart szacunek',
 'sąd patologia pierdolić',
 'jews kraść milion']

In [45]:
synphr_szan = get_synonymic_phrases(lemmphr_szan, stopwords=polish_stopwords,
                                    save_file='data/hateful_phrases/syn_szan.txt')
synphr_szan[:5]

['wypierdalać stąd , zaliczyć zgon',
 'kurwa , wypierdalać',
 'jaka odsunąć zamknąć pierdel',
 'jaka dalej tworzyć trudny sąd przejść ząłamanie nerwowy',
 'akta akt stół alba morda kubełl']

### Phrases appearance calculation

**How to calculate phrase appearance coefficient (PAC) in text?**

1. Split by whitespace lemmatized text and phrase to separate words.
2. Delete all stopwords and interpunction symbols from text and phrase.
3. For each word in phrase list all positions of phrase word in examined text. If no positions found, then omit this word.
4. Get all possible phrase words orders in examined text i.e. perform cartesian product for positions lists.
5. For each possible order:
    1. Form list of n positions into n-1 pairs.
    2. For each pair assign (1) if first element is smaller than second (ascending order) else (-1)
    3. Sum all assignations and divide the total by number of words in phrase (n) minus 1.
6. Return minimum, mean and maximum score.

**EXAMPLE 1.**:<br />
text: *Wróciły pisowskie trójki sądy doraźne koksowniki i SKOTy, do tego PiS gwałci żeby nie robić aborcji* <br />
phrase: *PiS gwałci żeby nie robić aborcji*<br />
<br />
Lemmatized text and phrase without stopwords:<br />
*wrócić(0) pisowski(1) trójka(2) sąd(3) doraźny(4) koksownik(5) skot(6) PiS(7) Pis(8) Pisa(9) gwałcić(10) żeby(11) robić(12) aborcja(13)*<br />
*PiS[7,] Pis[8,] Pisa[9,] gwałcić[10,] żeby[11,] robić[12,] aborcja[13,]*<br />
**n=7**<br />
<br />
Possible orders:<br />
--> (7, 8, 9, 10, 11, 12, 13): coeff=((+1) + (+1) + (+1) + (+1) + (+1) + (+1))/(7 - 1) = 1.0<br />
<br />
Results: **MIN=1.0 MEAN=1.0 MAX=1.0**




**EXAMPLE 2.**:<br />
text: *Faszystowskie sądy ach faszystowskie sądy*<br/>
phrase : *Ach faszystowskie sądy fałszywe*<br />
<br />
Lemmatized text and phrase without stopwords:<br />
*faszyzm(0) sąd(1) faszyzm(2) sąd(3)*<br />
*faszyzm[0, 2,] sąd[1, 3,] fałsz[]*<br />
**n=3**<br />
<br />
Possible orders:<br />
--> (0, 1): coeff=((+1))/(3-1)=0.5<br />
--> (0, 3): coeff=((+1))/(3-1)=0.5<br />
--> (2, 1): coeff=((-1))/(3-1)=-0.5<br />
--> (2, 3): coeff=((+1))/(3-1)=0.5<br />
<br />
Results: **MIN=-0.5 MEAN=0.25 MAX=0.5**

In [46]:
def calculate_PAC(text, phrase, lemmatized=False, stopwords=[]):
    
    t = text if lemmatized else lemmatize_text(text)
    p = phrase if lemmatized else lemmatize_text(phrase)
    
    t_words = list(filter(lambda x: x not in stopwords, t.split(' ')))
    p_words = list(filter(lambda x: x not in stopwords, p.split(' ')))
    
    assert (len(t_words) > 0), 'The examined text must have at least one non-stopword word!'
    
    if len(p_words) > 1:
        occurences = list([[i for i, x in enumerate(t_words) if x == p_w] for p_w in p_words])
        occurences = list([o for o in occurences if len(o) > 0])

        orders = list(itertools.product(*occurences))
        order_pairs_list = list([[tuple((o[i], o[i+1])) for i, oi in enumerate(o[:-1])] for o in orders])

        coeffs = list([sum([1. if op[0]<op[1] else -1. for op in ops])/(len(p_words) - 1)
                       for ops in order_pairs_list])

        return (np.min(coeffs), np.mean(coeffs), np.max(coeffs))
    elif len(p_words) == 1:
        return (1., 1., 1.) if p_words[0] in t_words else (0., 0., 0.)
    else:
        return (0., 0., 0.)

In [47]:
text = 'Wróciły pisowskie trójki sądy doraźne koksowniki i SKOTy, do tego PiS gwałci żeby nie robić aborcji'
phrase = 'PiS gwałci żeby nie robić aborcji'

calculate_PAC(text, phrase, stopwords=polish_stopwords)

(1.0, 1.0, 1.0)

In [48]:
text = 'Faszystowskie sądy ach faszystowskie sądy'
phrase = 'Ach faszystowskie sądy fałszywe'

calculate_PAC(text, phrase, stopwords=polish_stopwords)

(-0.5, 0.25, 0.5)

**Calculate PAC score for all tweets.**

1. Load relevant data with sanitized tweets with classes and all hateful phrases.
2. For each tweet:
    1. For each hate type:
        1. Calculate PAC scores (min, mean, max) for every phrase which belongs to certain hate type
        2. Get means of minimum, mean and maximum PAC scores
        3. Write calculations into dictionary
    2. Write all hate types dictionary values into .csv row.

In [49]:
FULL_HATE_TYPES = ['wyzywanie', 'grożenie', 'wykluczanie', 'odczłowieczanie', 'poniżanie',
                   'stygmatyzacja', 'szantaż']
HATE_TYPES = ['wyz', 'groz', 'wyk', 'odcz', 'pon', 'styg', 'szan']

In [50]:
lemmatized_phrases = list([
    lemmphr_wyz, lemmphr_groz, lemmphr_wyk, lemmphr_odcz, lemmphr_pon, lemmphr_styg, lemmphr_szan
])
synonymic_phrases = list([
    synphr_wyz, synphr_groz, synphr_wyk, synphr_odcz, synphr_pon, synphr_styg, synphr_szan
])

In [51]:
df_data = df_combined[['id', 'tweet', 'wyzywanie', 'grożenie', 'wykluczanie', 'odczłowieczanie', 'poniżanie',
                       'stygmatyzacja', 'szantaż']]
ids  = df_data['id']
tweets = df_data['tweet']
df_data = df_data.notnull().astype('int')
df_data['id'] = ids
df_data['tweet'] = tweets
del ids, tweets
df_data.head(2)

,id,tweet,wyzywanie,grożenie,wykluczanie,odczłowieczanie,poniżanie,stygmatyzacja,szantaż
0,0,Dokładnie! Dlatego trzeba komuchów gonić przed sądy póki żyją. I mięć otwarte oczy na komuchów zakamuflowanych,1,0,0,0,0,0,0
1,1,Polska Polityka: Sądy bardziej bezkarne niż w PRL,0,0,0,0,0,0,0


In [52]:
lemm_tweets = list([lemmatize_text(tweet) for tweet in df_data['tweet']])
df_data['lemmatized'] = lemm_tweets
del lemm_tweets
df_data.head(2)

,id,tweet,wyzywanie,grożenie,wykluczanie,odczłowieczanie,poniżanie,stygmatyzacja,szantaż,lemmatized
0,0,Dokładnie! Dlatego trzeba komuchów gonić przed sądy póki żyją. I mięć otwarte oczy na komuchów zakamuflowanych,1,0,0,0,0,0,0,dokładnie ! dlatego trzeba komuch gonić przed sąd póki żyć . i miąć otworzyć otwarty oko oczyć na komuch zakamuflować
1,1,Polska Polityka: Sądy bardziej bezkarne niż w PRL,0,0,0,0,0,0,0,polska polski polityka polityk sąd bardzo bezkarny niż niża niżyć nizać niż w prl


In [53]:
PAC_SCORES_PATH = 'data/sady_main/sady_pac_scores.csv'

In [54]:
if not os.path.exists(PAC_SCORES_PATH):
    with open(PAC_SCORES_PATH, 'w') as f:
        csv.writer(f).writerow([
            'id', 'tweet',
            'wyz_PAC_min', 'wyz_PAC_mean', 'wyz_PAC_max', 'wyz_label',
            'groz_PAC_min', 'groz_PAC_mean', 'groz_PAC_max', 'groz_label',
            'wyk_PAC_min', 'wyk_PAC_mean', 'wyk_PAC_max', 'wyk_label',
            'odcz_PAC_min', 'odcz_PAC_mean', 'odcz_PAC_max', 'odcz_label', 
            'pon_PAC_min', 'pon_PAC_mean', 'pon_PAC_max', 'pon_label',
            'styg_PAC_min', 'styg_PAC_mean', 'styg_PAC_max', 'styg_label',
            'szan_PAC_min', 'szan_PAC_mean', 'szan_PAC_max', 'szan_label',
        ])

    for _id, tweet in tqdm(df_data.iterrows(), total=len(df_data)):
        scores = dict({})
        
        for hate_type, l_phrases, s_phrases in zip(HATE_TYPES, lemmatized_phrases, synonymic_phrases):
            sc_min, sc_mean, sc_max = list([]), list([]), list([])

            for l_phrase in l_phrases:
                mn, mean, mx = calculate_PAC(tweet['lemmatized'], l_phrase, lemmatized=True,
                                             stopwords=polish_stopwords)
                sc_min.append(mn)
                sc_mean.append(mean)
                sc_max.append(mx)

            for s_phrase in s_phrases:
                mn, mean, mx = calculate_PAC(tweet['lemmatized'], s_phrase, lemmatized=True,
                                             stopwords=polish_stopwords)
                sc_min.append(mn)
                sc_mean.append(mean)
                sc_max.append(mx)

            scores[f'{hate_type}_min'] = np.min(sc_min)
            scores[f'{hate_type}_mean'] = np.mean(sc_mean)
            scores[f'{hate_type}_max'] = np.max(sc_max)
            del sc_min, sc_mean, sc_max
        
        with open(PAC_SCORES_PATH, 'a') as f:
            csv.writer(f).writerow([
                _id, tweet['tweet'],
                scores['wyz_min'], scores['wyz_mean'], scores['wyz_max'], tweet['wyzywanie'],
                scores['groz_min'], scores['groz_mean'], scores['groz_max'], tweet['grożenie'],
                scores['wyk_min'], scores['wyk_mean'], scores['wyk_max'], tweet['wykluczanie'],
                scores['odcz_min'], scores['odcz_mean'], scores['odcz_max'], tweet['odczłowieczanie'],
                scores['pon_min'], scores['pon_mean'], scores['pon_max'], tweet['poniżanie'],
                scores['styg_min'], scores['styg_mean'], scores['styg_max'], tweet['stygmatyzacja'],
                scores['szan_min'], scores['szan_mean'], scores['szan_max'], tweet['szantaż'],
            ])
        del scores

In [55]:
df_pac_scores = pd.read_csv(PAC_SCORES_PATH)
df_pac_scores.head(2)

,id,tweet,wyz_PAC_min,wyz_PAC_mean,wyz_PAC_max,wyz_label,groz_PAC_min,groz_PAC_mean,groz_PAC_max,groz_label,...,pon_PAC_max,pon_label,styg_PAC_min,styg_PAC_mean,styg_PAC_max,styg_label,szan_PAC_min,szan_PAC_mean,szan_PAC_max,szan_label
0,0,Dokładnie! Dlatego trzeba komuchów gonić przed sądy póki żyją. I mięć otwarte oczy na komuchów zakamuflowanych,-0.000129,0.002574,0.005277,1,-0.002833,-0.002833,-0.002833,0,...,0.000440,0,-0.000614,0.000698,0.002010,0,0.0,0.0,0.0,0
1,1,Polska Polityka: Sądy bardziej bezkarne niż w PRL,0.000000,0.000000,0.000000,0,-0.000527,-0.000527,-0.000527,0,...,-0.004546,0,0.000097,0.000097,0.000097,0,0.0,0.0,0.0,0


## Polish Polyglot sentiment analysis

In [56]:
def text_sentiment(text):
    
    # detect and delete invalid characters first
    t = text
    invalid = set()
    for i, ch in enumerate(t):
        try:
            Text(f"Char: {ch}").words
        except:
            invalid.add(ch)
    for ch in invalid:
        t = t.replace(ch, '')
    
    t = Text(t)
    sents = list([])
    for w in t.words:
        try:
            s = w.polarity
        except ValueError:
            s = 0
        sents.append(s)
    sents = np.array(sents)
    
    return np.size(sents[sents==-1]), np.size(sents[sents==0]), np.size(sents[sents==1])

In [57]:
text_sentiment('Wróciły pisowskie trójki sądy doraźne koksowniki i SKOTy, do tego PiS gwałci żeby nie robić aborcji')

(0, 17, 0)

In [58]:
text_sentiment('Faszystowskie sądy ach faszystowskie sądy')

(0, 5, 0)

## Characters, syllables, words counting

In [59]:
'pl_PL' in pyphen.LANGUAGES

True

In [60]:
dic = pyphen.Pyphen(lang='pl_PL')

In [61]:
def text_numbers(text):
    num_chars = len(text.replace(' ', ''))
    num_syllables = sum([len(dic.inserted(word).split('-')) for word in text.split(' ')])
    num_words = len(text.split(' '))
    num_unique_words = len(set(text.lower().split(' ')))
    
    return num_chars, num_syllables, num_words, num_unique_words

In [62]:
text_numbers('Wróciły pisowskie trójki sądy doraźne koksowniki i SKOTy, do tego PiS gwałci żeby nie robić aborcji')

(84, 33, 16, 16)

In [63]:
text_numbers('Faszystowskie sądy ach faszystowskie sądy')

(37, 13, 5, 3)

In [64]:
text_numbers(lemmatize_text('Wróciły pisowskie trójki sądy doraźne koksowniki i SKOTy, do tego PiS gwałci żeby nie robić aborcji'))

(90, 34, 21, 20)

In [65]:
text_numbers(lemmatize_text('Faszystowskie sądy ach faszystowskie sądy'))

(33, 11, 5, 3)

**Calculate above other scores for all tweets.**

1. Load relevant data with sanitized tweets.
2. For each tweet:
    1. Remove invalid (for polyglot) characters which cause errors.
    2. Determine how many words have which of three sentiment types.
    3. Count characters, syllables, words and unique words.
    2. Write all values into .csv row.

In [66]:
OTHER_SCORES_PATH = 'data/sady_main/sady_other_scores.csv'

In [67]:
if not os.path.exists(OTHER_SCORES_PATH):
    with open(OTHER_SCORES_PATH, 'w') as f:
        csv.writer(f).writerow([
            'id', 'tweet',
            's_neg', 's_neu', 's_pos',
            'n_chars', 'n_sylls', 'n_words', 'nu_words',
            'nl_chars', 'nl_sylls', 'nl_words', 'nlu_words',
        ])

    for _id, tweet in tqdm(df_data.iterrows(), total=len(df_data)):
        scores = dict({})
        
        scores['neg'], scores['neu'], scores['pos'] = text_sentiment(tweet['tweet'])
        scores['chars'], scores['sylls'], scores['words'], scores['u_words'] = text_numbers(tweet['tweet'])
        scores['l_chars'], scores['l_sylls'], scores['l_words'], scores['l_u_words'] = text_numbers(tweet['lemmatized'])
        
        with open(OTHER_SCORES_PATH, 'a') as f:
            csv.writer(f).writerow([
                _id, tweet['tweet'],
                scores['neg'], scores['neu'], scores['pos'],
                scores['chars'], scores['sylls'], scores['words'], scores['u_words'],
                scores['l_chars'], scores['l_sylls'], scores['l_words'], scores['l_u_words'],
            ])
        del scores

In [68]:
df_other_scores = pd.read_csv(OTHER_SCORES_PATH)
df_other_scores.head(2)

,id,tweet,s_neg,s_neu,s_pos,n_chars,n_sylls,n_words,nu_words,nl_chars,nl_sylls,nl_words,nlu_words
0,0,Dokładnie! Dlatego trzeba komuchów gonić przed sądy póki żyją. I mięć otwarte oczy na komuchów zakamuflowanych,0,18,0,95,36,17,16,98,35,20,19
1,1,Polska Polityka: Sądy bardziej bezkarne niż w PRL,0,9,0,42,17,9,9,68,28,15,14


## Hateful phrases topics detection

**Find top 5 topic sentences for pharases of each hate type.**

1. For each hate type:
    1. Get relevant lemmatized and synonymic phrases and combine them into one list.
    2. Fit CountVectorizer and LDA model.
    3. Save trained model into pickle archive.
    4. For each tweet:
        1. Calculate PAC scores of each of 10 topics appearance.
        2. Save into .csv file.

In [69]:
N_TOPICS, N_WORDS = 10, 10

In [70]:
for hate_type, l_phrases, s_phrases in tqdm(zip(HATE_TYPES, lemmatized_phrases, synonymic_phrases), total=len(HATE_TYPES)):
    
    cv = CountVectorizer(stop_words=polish_stopwords)
    count_data = cv.fit_transform(l_phrases + s_phrases)
    
    lda_model = LDA(n_components=N_TOPICS, n_jobs=-1)
    lda_model.fit(count_data)
    
    with open(f'models/lda/lda_{hate_type}.pkl', 'wb') as f:
        pickle.dump([lda_model, cv], f)

In [71]:
def lda_topics(lda_model, lda_cv, n_words):
    words = lda_cv.get_feature_names()
    
    topics = list([' '.join([words[i] for i in topic.argsort()[:-n_words - 1:-1]])
                   for topic in lda_model.components_])
    
    return topics

In [72]:
TOPIC_PAC_SCORES_PATH = 'data/sady_main/sady_topic_pac_scores.csv'

In [73]:
if not os.path.exists(TOPIC_PAC_SCORES_PATH):
    with open(TOPIC_PAC_SCORES_PATH, 'w') as f:
        csv.writer(f).writerow([
            'id', 'tweet',
            'wyz_min', 'wyz_mean', 'wyz_max',
            'groz_min', 'groz_mean', 'groz_max',
            'wyk_min', 'wyk_mean', 'wyk_max',
            'odcz_min', 'odcz_mean', 'odcz_max', 
            'pon_min', 'pon_mean', 'pon_max',
            'styg_min', 'styg_mean', 'styg_max',
            'szan_min', 'szan_mean', 'szan_max',
            'vulg_min', 'vulg_mean', 'vulg_max',
        ])
    
    for _id, tweet in tqdm(df_data.iterrows(), total=len(df_data)):
        scores = dict({})
        
        for hate_type in HATE_TYPES + ['vulg']:
            with open(f'models/lda/lda_{hate_type}.pkl', 'rb') as f:
                lda_model, cv = pickle.load(f)

            topics = lda_topics(lda_model, cv, n_words=N_WORDS)
            sc_min, sc_mean, sc_max = list([]), list([]), list([])

            for topic in topics:
                mn, mean, mx = calculate_PAC(tweet['lemmatized'], topic, lemmatized=True,
                                             stopwords=polish_stopwords)
                sc_min.append(mn)
                sc_mean.append(mean)
                sc_max.append(mx)
            
            scores[f'{hate_type}_min'] = np.min(sc_min)
            scores[f'{hate_type}_mean'] = np.mean(sc_mean)
            scores[f'{hate_type}_max'] = np.max(sc_max)
            del sc_min, sc_mean, sc_max
            
        with open(TOPIC_PAC_SCORES_PATH, 'a') as f:
            csv.writer(f).writerow([
                _id, tweet['tweet'],
                scores['wyz_min'], scores['wyz_mean'], scores['wyz_max'],
                scores['groz_min'], scores['groz_mean'], scores['groz_max'],
                scores['wyk_min'], scores['wyk_mean'], scores['wyk_max'],
                scores['odcz_min'], scores['odcz_mean'], scores['odcz_max'],
                scores['pon_min'], scores['pon_mean'], scores['pon_max'],
                scores['styg_min'], scores['styg_mean'], scores['styg_max'],
                scores['szan_min'], scores['szan_mean'], scores['szan_max'],
                scores['vulg_min'], scores['vulg_mean'], scores['vulg_max'],
            ])
            
        del scores

In [74]:
df_topic_pac_scores = pd.read_csv(TOPIC_PAC_SCORES_PATH)
df_topic_pac_scores.head(2)

,id,tweet,wyz_min,wyz_mean,wyz_max,groz_min,groz_mean,groz_max,wyk_min,wyk_mean,...,pon_max,styg_min,styg_mean,styg_max,szan_min,szan_mean,szan_max,vulg_min,vulg_mean,vulg_max
0,0,Dokładnie! Dlatego trzeba komuchów gonić przed sądy póki żyją. I mięć otwarte oczy na komuchów zakamuflowanych,-0.111111,0.000000,0.111111,0.000000,0.0,0.000000,0.0,0.000000,...,0.0,0.000000,0.011111,0.111111,0.0,0.0,0.0,0.0,0.0,0.0
1,1,Polska Polityka: Sądy bardziej bezkarne niż w PRL,0.000000,0.011111,0.111111,-0.111111,0.0,0.111111,0.0,0.033333,...,0.0,-0.111111,-0.022222,0.000000,0.0,0.0,0.0,0.0,0.0,0.0
